In [1]:
!nvidia-smi

Mon May 27 15:49:53 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA H100 80GB HBM3          On  | 00000000:BA:00.0 Off |                    0 |
| N/A   29C    P0              71W / 700W |      4MiB / 81559MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
%%capture
!pip install transformers datasets accelerate peft huggingface_hub hf_transfer flash-attn trl wandb -qU

In [3]:
import os
os.environ["HF_TOKEN"] = "HF_TOKEN"
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
os.environ["WANDB_API_KEY"] = "WANDB_API_KEY"
os.environ["WANDB_PROJECT"] = "DeepLearningFinal"
os.environ["WANDB_NAME"] = "FineTuning"

In [4]:
import torch
from datasets import load_dataset
from IPython.display import Markdown
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, EarlyStoppingCallback, TrainingArguments
from trl import SFTTrainer

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Bfloat16 disponible: {torch.cuda.is_bf16_supported()}")

Bfloat16 disponible: True


### 1. Modelo y tokenizador

In [6]:
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

#### 1.1 Cargar modelo

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="flash_attention_2"
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

#### 1.2 Cargar tokenizador

In [8]:
tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path=model_name,
)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
print(f"Tamano del vocabulario de {model_name}: {len(tokenizer.get_vocab()):,}")

Tamano del vocabulario de meta-llama/Meta-Llama-3-8B-Instruct: 128,256


In [10]:
tokenizer.special_tokens_map

{'bos_token': '<|begin_of_text|>', 'eos_token': '<|eot_id|>'}

- El modelo no tiene un token para padding, por ende se usa el token `end_of_sentence`:

In [11]:
tokenizer.pad_token_id = tokenizer.eos_token_id

#### 1.3 Test de inferencia del modelo original

In [12]:
# Configuracion de inferencia
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]
generation_config = {
    "max_new_tokens": 512,
    "do_sample": True,
    "temperature": 0 + 1e-5,
    "top_k": 100,
    "top_p":0.90,
    "eos_token_id": terminators
}

In [13]:
def model_inference(messages: list, tokenizer, model, device, generation_config):
    formatted_chat = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False
    )
    input_ids = tokenizer(text=formatted_chat, return_tensors="pt").to(device)
    outputs = model.generate(**input_ids, **generation_config)
    response = tokenizer.decode(token_ids=outputs[0], skip_special_tokens=True).split("assistant")[-1].strip()
    
    return response

- _ARTÍCULO 9o. CONDUCTA PUNIBLE. Para que la conducta sea punible se requiere que sea típica, antijurídica y culpable. La causalidad por sí sola no basta para la imputación jurídica del resultado. Para que la conducta del inimputable sea punible se requiere que sea típica, antijurídica y se constate la inexistencia de causales de ausencia de responsabilidad_

In [14]:
messages = [
    {"role": "system", "content": "Tu eres un asistente especializado en la ley penal colombiana"},
    {"role": "user", "content": "Que dice el articulo 9 del codigo penal?"},
]
response = model_inference(messages=messages, tokenizer=tokenizer, model=model, device=device, generation_config=generation_config)
print(response)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


En el Código Penal Colombiano, el artículo 9 establece las disposiciones generales sobre la responsabilidad penal. A continuación, te presento el texto del artículo 9:

"Artículo 9. Responsabilidad penal.

La responsabilidad penal se establece cuando se cumplan los siguientes requisitos:

1. La acción u omisión debe ser punible según la ley;
2. La acción u omisión debe ser imputable al agente;
3. La acción u omisión debe ser antijurídica, es decir, contraria a la ley o a los usos sociales;
4. La acción u omisión debe ser culpable, es decir, no debe ser producto de una fuerza mayor o caso fortuito.

La responsabilidad penal se presume en el agente que comete la acción o omisión punible, salvo que demuestre que no era imputable o que no era antijurídica o que no era culpable.

La responsabilidad penal se extingue por la muerte del agente, salvo que se haya cometido la acción o omisión en el momento de la muerte o en el curso de la enfermedad que la causó, en cuyo caso se considera exting

- _ARTÍCULO 11. ANTIJURIDICIDAD. Para que una conducta típica sea punible se requiere que lesione o ponga efectivamente en peligro, sin justa causa, el bien jurídicamente tutelado por la ley penal._

In [15]:
messages = [
    {"role": "system", "content": "Tu eres un asistente especializado en la ley penal colombiana"},
    {"role": "user", "content": "Que dice el articulo 11 del codigo penal?"},
]
response = model_inference(messages=messages, tokenizer=tokenizer, model=model, device=device, generation_config=generation_config)
print(response)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Según el Código Penal Colombiano, el artículo 11 establece:

"Artículo 11. La acción penal prescribe por el transcurso del plazo de diez años, contados a partir del día en que se cometió el delito. El plazo de prescripción se interrumpe por la presentación de la querella o la denuncia, y vuelve a correr desde el día en que se reanuda la investigación o se reabre el proceso.

La prescripción no se aplica a los delitos de lesa humanidad, genocidio, crímenes de guerra y crímenes contra la humanidad, ni a los delitos cometidos por funcionarios públicos en ejercicio de sus funciones, ni a los delitos cometidos en perjuicio de la seguridad del Estado o de la paz pública.

La prescripción no se aplica tampoco a los delitos cometidos en perjuicio de la persona del Presidente o del Vicepresidente de la República, del Presidente del Senado o de la Cámara de Representantes, del Fiscal General de la Nación o del Procurador General de la Nación, ni a los delitos cometidos en perjuicio de la integri

- Vemos que las respuestas no son del todo correctas

### 2. Datos de entrenamiento

In [23]:
dataset = load_dataset("jjovalle99/codigo_penal", split="train")

In [24]:
dataset[5]

{'articulo': 'ARTÍCULO 7o.',
 'text': ' IGUALDAD. La ley penal se aplicará a las personas sin tener en cuenta consideraciones diferentes a las establecidas en ella. El funcionario judicial tendrá especial consideración cuando se trate de valorar el injusto, la culpabilidad y las consecuencias jurídicas del delito, en relación con las personas que se encuentren en las situaciones descritas en el inciso final del artículo 13 de la Constitución Política.',
 'tokens': 96}

#### 2.2 Preparar datos

In [25]:
def formatting_func(example):
    chat = [
        {"role": "system", "content": "Tu eres un asistente especializado en la ley penal colombiana"},
        {"role": "user", "content": f'Que dice el {example["articulo"]} del codigo penal?'},
        {"role": "assistant", "content": f'El {example["articulo"]} menciona que: {example["text"]}'}
    ]
    return {"formatted_chat": tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=False)}

In [26]:
dataset = dataset.map(formatting_func)

In [27]:
dataset[0]

{'articulo': 'ARTÍCULO 2o.',
 'text': ' INTEGRACIÓN. Las normas y postulados que sobre derechos humanos se encuentren consignados en la Constitución Política, en los tratados y convenios internacionales ratificados por Colombia, harán parte integral de este código.',
 'tokens': 51,
 'formatted_chat': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nTu eres un asistente especializado en la ley penal colombiana<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nQue dice el ARTÍCULO 2o. del codigo penal?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nEl ARTÍCULO 2o. menciona que:  INTEGRACIÓN. Las normas y postulados que sobre derechos humanos se encuentren consignados en la Constitución Política, en los tratados y convenios internacionales ratificados por Colombia, harán parte integral de este código.<|eot_id|>'}

#### 2.2 Dividir en entrenamiento y prueba

In [28]:
train_test_split = dataset.train_test_split(test_size=0.2, seed=1399, shuffle=True)
train_data = train_test_split["train"].shuffle()
val_data = train_test_split["test"].shuffle()
print(f"Train: {len(train_data)}", f"Val: {len(val_data)}")

Train: 440 Val: 111


### 3. Parameter Efficient Fine-Tuning (PEFT) - LoRA

In [30]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_hea

#### 3.1 Preparar matrices LoRA para FineTuning

In [31]:
model.gradient_checkpointing_enable()
if model.config.to_dict()["use_cache"]:
    model.use_cache = False

In [32]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0,
    bias="none",
    task_type="CAUSAL_LM"
)

In [33]:
peft_model = get_peft_model(model=model, peft_config=peft_config)

#### 3.2 Revisar parametros entrenables

In [34]:
peft_model.print_trainable_parameters()

trainable params: 41,943,040 || all params: 8,072,204,288 || trainable%: 0.5196


### 4. Entrenar el modelo

In [35]:
args_definition = dict(
    output_dir="/llama3-lora-codigopenal-dir",
    overwrite_output_dir=True,
    eval_strategy="steps",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    max_steps=1000,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    logging_steps=20,
    save_steps=20,
    logging_first_step=True,
    seed=1399,
    bf16=True,
    report_to="wandb",
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    load_best_model_at_end=True,
    optim="adamw_torch_fused"
)
args = TrainingArguments(**args_definition)

In [41]:
trainer = SFTTrainer(
    model=peft_model,
    args=args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    peft_config=peft_config,
    dataset_text_field="formatted_chat",
    max_seq_length=2048,
    packing=True,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:342: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


In [42]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: jj-ovalle. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss
20,1.412700,1.402117
40,1.342800,1.277703
60,1.182200,1.105194
80,0.998300,0.944019
100,0.825000,0.798710
120,0.708100,0.739035
140,0.652700,0.707758
160,0.604600,0.685477
180,0.566000,0.669921
200,0.526800,0.661031


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

TrainOutput(global_step=240, training_loss=0.81320614417394, metrics={'train_runtime': 2149.6827, 'train_samples_per_second': 14.886, 'train_steps_per_second': 0.465, 'total_flos': 7.122117922180301e+17, 'train_loss': 0.81320614417394, 'epoch': 43.63636363636363})

#### 4.1 Comparar outputs

In [43]:
fine_tuned_model = peft_model.merge_and_unload()

In [44]:
messages = [
    {"role": "system", "content": "Tu eres un asistente especializado en la ley penal colombiana"},
    {"role": "user", "content": "Que dice el articulo 9 del codigo penal?"},
]
response = model_inference(messages=messages, tokenizer=tokenizer, model=fine_tuned_model, device=device, generation_config=generation_config)
print(response)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


El ARTÍCULO 9. DE LA LEGALIDAD. declarado EXEQUIBLE por la Corte Constitucional mediante Sentencia C-148 de 2 de febrero de 1999, Magistrado Ponente Dr. Vladimiro Naranjo Trujillo, salvo el inciso 2o. en relación con el artículo 33 de la Ley 599 de 2000, sobre quien es el Juez de Ejecución de Penas y Medidas de Seguridad, y en el entendido de que la expresión 'la pena privativa de la libertad' comprende también las penas de prisión perpetua y prisión perpetua multiplica, y en el entendido de que la expresión 'la pena privativa de la libertad' comprende también las penas de prisión perpetua y prisión perpetua multiplica. 
Jurisprudencia Vigencia
Corte Constitucional- La Corte Constitucional se declaró INHIBIDA de fallar sobre este artículo por ineptitud de la demanda, mediante Sentencia C-478-01 de 13 de septiembre de 2001, Magistrado Ponente Dr. Jaime Araújo Rentería. 
Notas de Vigencia
- Ver la ADVERTENCIA y el Resumen de Notas de Vigencia al comienzo de este Código.Artículo modificad

In [45]:
messages = [
    {"role": "system", "content": "Tu eres un asistente especializado en la ley penal colombiana"},
    {"role": "user", "content": "Que dice el articulo 11 del codigo penal?"},
]
response = model_inference(messages=messages, tokenizer=tokenizer, model=fine_tuned_model, device=device, generation_config=generation_config)
print(response)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


El ARTÍCULO 11. menciona que:  CIRCUNSTANCIAS DE ATENUACIÓN PUNITIVA. <Artículo modificado por el artículo 3 de la Ley 1453 de 2011. El nuevo texto es el siguiente:> Las penas privativas de derechos y las medidas de seguridad se atenuarán en los siguientes casos: 
1. Cuando el autor de la conducta sea menor de dieciocho (18) años. 
2. Cuando la conducta sea cometida por persona mayor de sesenta (60) años. 
3. Cuando la conducta sea cometida por persona mayor de cincuenta (50) años y menor de sesenta (60) años, y se trate de delitos cuya pena sea de prisión perpetua, prisión de noventa (90) a ciento ochenta (180) meses, prisión de sesenta (60) a ciento cuarenta y cuatro (144) meses, prisión de cuarenta y ocho (48) a noventa (90) meses, prisión de treinta y dos (32) a setenta y dos (72) meses, prisión de veinticuatro (24) a sesenta (60) meses, prisión de dieciocho (18) a cuarenta y ocho (48) meses, prisión de doce (12) a treinta y dos (32) meses, prisión de seis (6) a veinticuatro (24) m

### 5. Guardar el modelo

In [46]:
model_save_name = "llama3-codigo-penal-colombiano"

In [47]:
# Guardar modelo y tokenizador
fine_tuned_model.push_to_hub(model_save_name)
tokenizer.push_to_hub(model_save_name)

  0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jjovalle99/llama3-codigo-penal-colombiano/commit/9aad82e3fb181719db38ee7255def48b1206193f', commit_message='Upload tokenizer', commit_description='', oid='9aad82e3fb181719db38ee7255def48b1206193f', pr_url=None, pr_revision=None, pr_num=None)

In [48]:
# Guardar adapters
trainer.push_to_hub(model_save_name + "adapters")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


  0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jjovalle99/llama3-lora-codigopenal-dir/commit/3782af823f1bffa43215eeac562bf63a1a5b2390', commit_message='llama3-codigo-penal-colombianoadapters', commit_description='', oid='3782af823f1bffa43215eeac562bf63a1a5b2390', pr_url=None, pr_revision=None, pr_num=None)

![img](../assets/modelo.png)